In [1]:
import tqdm
import collections
import more_itertools
import wandb
import pandas as pd
import torch
import random
import string

# Set device for torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
torch.manual_seed(42)
random.seed(52)

# Define possible repeat lengths (e.g., 'aaa', 'bbbb', 'ccccccc')
repeats_range = [3, 4, 5, 7]

# Generate `text8` with variable-length sequences for each letter in the alphabet
text8 = ' '.join([
    ' '.join([char] * random.choice(repeats_range)) for char in string.ascii_lowercase
])

# Print the generated `text8`
print(text8)


a a a a a b b b c c c c c c c d d d d d e e e e e e e f f f g g g g h h h h i i i i i i i j j j j k k k k k l l l l l l l m m m n n n o o o o o o o p p p p p p p q q q r r r r s s s t t t t u u u u v v v v v v v w w w w x x x x x y y y y y y y z z z z z z z


In [2]:
def preprocess(text: str) -> list[str]:
  text = text.lower()
  words = text.split()
  stats = collections.Counter(words)
  words = [word for word in words if stats[word] > 0]
  return words

In [1]:
import tqdm
import collections
import more_itertools
import wandb
import pandas as pd
import torch
import random
import string


def preprocess(text: str) -> list[str]:
  text = text.lower()
  text = text.replace('.',  ' <PERIOD> ')
  text = text.replace(',',  ' <COMMA> ')
  text = text.replace('"',  ' <QUOTATION_MARK> ')
  text = text.replace(';',  ' <SEMICOLON> ')
  text = text.replace('!',  ' <EXCLAMATION_MARK> ')
  text = text.replace('?',  ' <QUESTION_MARK> ')
  text = text.replace('(',  ' <LEFT_PAREN> ')
  text = text.replace(')',  ' <RIGHT_PAREN> ')
  text = text.replace('--', ' <HYPHENS> ')
  text = text.replace('?',  ' <QUESTION_MARK> ')
  text = text.replace(':',  ' <COLON> ')
  words = text.split()
  stats = collections.Counter(words)
  words = [word for word in words if stats[word] > 5]
  return words

In [2]:
with open('text8') as f: text8: str = f.read()
titles_string = ' '.join(text8)  # Joining with a space

# Concatenate the titles string to the text8 variable
text8 += ' ' + titles_string  # Add a space for separation

In [3]:
corpus: list[str] = preprocess(text8)

In [5]:
print(text8[:25])

 anarchism originated as 


In [6]:
def create_lookup_tables(words: list[str]) -> tuple[dict[str, int], dict[int, str]]:
  word_counts = collections.Counter(words)
  vocab = sorted(word_counts, key=lambda k: word_counts.get(k), reverse=True)
  int_to_vocab = {ii+1: word for ii, word in enumerate(vocab)}
  int_to_vocab[0] = '<PAD>'
  vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
  return vocab_to_int, int_to_vocab

In [7]:
#creating dictionary 
words_to_ids, ids_to_words = create_lookup_tables(corpus)

In [8]:
words_to_ids['anarchism']

5234

In [9]:
#creating token for the list we have from our dictionary 
tokens = [words_to_ids[word] for word in corpus]

In [10]:
import torch

class SkipGramFoo(torch.nn.Module):
    def __init__(self, voc, emb, ctx):
        super().__init__()
        self.emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)
        self.ctx_emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)  # Additional embedding for context
        self.ffw = torch.nn.Linear(in_features=emb, out_features=voc, bias=False)
        #self.sig = torch.nn.Sigmoid()

    def forward(self, inpt, trgs):
        
        # Lets stack these tensors
        # input is the tar token 1x1
        # trgs is the cw 1x4
        print("this is the shape inpt",inpt.shape) #[1]
        print("this is the shape trgs",trgs.shape) # [1,4]
        
        # need to unsqueeze
        inpt = inpt.unsqueeze(1)

        print("this is the shape new inpt",inpt.shape) #[1,1]

        cw_tensor = torch.cat((inpt,trgs) ,dim=1)
        
        print("this is the shape",cw_tensor.shape)
        
        #Get embeddings for input and context
        emb = self.emb(cw_tensor)  # Shape: (batch_size, emb_dim) -> e.g., (1, 64)
        print('shape of emb',emb.shape)

        ctx = self.ctx_emb(cw_tensor)  # Shape: (batch_size, num_context_tokens, emb_dim) -> e.g., (1, 4, 64)
        print('shape of target words',ctx.shape)
        
        ctx = ctx.squeeze(0)
        
        print('shape of ctx',ctx.shape)

        
        similarity_matrix = torch.matmul(ctx, ctx.T) 
        print('similarity matrix is', similarity_matrix.shape)

        
        soft_matrix = torch.nn.functional.softmax(similarity_matrix, dim=1)
        
        print("softmax matrix shape",soft_matrix.shape)
        
        attention = torch.matmul(soft_matrix, emb)  #Attention embeddings
        
        print("attention", attention.shape)

        
        out = self.ffw(attention[:, 0, :])  # project this
        
        # out = torch.nn.functional.softmax(out, dim=1)
        print("show me what out looks like -->", out)
        

        # out = self.sig(out).clamp(min=1e-7, max=1 - 1e-7)

    
        return out

In [20]:
import torch

class SkipGramFoo(torch.nn.Module):
    def __init__(self, voc, emb, ctx):
        super().__init__()
        self.emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)
        self.ctx_emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)  # Additional embedding for context
        self.ffw = torch.nn.Linear(in_features=emb, out_features=voc, bias=False)
        
        # Add learnable bias for the attention embeddings
        self.attn_embedding_bias = torch.nn.Parameter(torch.zeros(emb))

    def forward(self, inpt, trgs):
        # Stack input and target tensors
        # print("this is the shape inpt", inpt.shape)  # [1]
        # print("this is the shape trgs", trgs.shape)  # [1,4]
        
        # Unsqueeze input
        inpt = inpt.unsqueeze(1)
        # print("this is the shape new inpt", inpt.shape)  # [1,1]
        
        # Concatenate input and targets
        cw_tensor = torch.cat((inpt, trgs), dim=1)
        # print("this is the shape", cw_tensor.shape)
        
        # Get embeddings for input and context
        emb = self.emb(cw_tensor)  # Shape: (batch_size, emb_dim) -> e.g., (1, 64)
        # print('shape of emb', emb.shape)
        
        ctx = self.ctx_emb(cw_tensor)  # Shape: (batch_size, num_context_tokens, emb_dim) -> e.g., (1, 4, 64)
        # print('shape of target words', ctx.shape)
        
        # Squeeze context embeddings
        ctx = ctx.squeeze(0)
        # print('shape of ctx', ctx.shape)
        
        # Calculate similarity matrix and apply softmax for attention
        similarity_matrix = torch.matmul(ctx, ctx.T)
        # print('similarity matrix is', similarity_matrix.shape)
        
        soft_matrix = torch.nn.functional.softmax(similarity_matrix, dim=1)
        # print("softmax matrix shape", soft_matrix.shape)
        
        # Calculate the attention-weighted embeddings and add the bias
        attention = torch.matmul(soft_matrix, emb) + self.attn_embedding_bias  # Add bias here
        # print("attention with bias", attention.shape)
        
        # Pass through linear layer
        out = self.ffw(attention[:, 0, :])  # Project to output vocabulary space
        # print("show me what out looks like -->", out)
        
        return out


In [19]:
args = (len(words_to_ids), 64,2)
mFoo = SkipGramFoo(*args)
print('mFoo', sum(p.numel() for p in mFoo.parameters()))

mFoo 12219328


In [12]:
#learning rate thing 
opFoo = torch.optim.Adam(mFoo.parameters(), lr=0.003)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
import more_itertools
import tqdm
import wandb

# Initialize W&B
wandb.init(project="word2vec_attention", entity="omareweis123", name='bias weighting')

# Set parameters
learning_rate = 0.001  # Define your learning rate
mFoo = mFoo.to(device)

# Set context size
context_size = 2  # Example context size
window_size = 2 * context_size + 1  # Total tokens in the window

# Initialize the optimizer
opFoo = torch.optim.Adam(mFoo.parameters(), lr=learning_rate)

## Instantiate the CrossEntropyLoss
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(30):
    wins = list(more_itertools.windowed(tokens[:100], window_size))  # Convert to list for easier iteration
    prgs = tqdm.tqdm(wins, total=len(wins), desc=f"Epoch {epoch + 1}", leave=False)

    total_loss = 0.0  # Initialize total loss for the epoch

    for win in prgs:
        # Prepare input and target tensors for a single window
        inpt = torch.LongTensor([win[context_size]]).to(device)  # Central token for the window
        trgs = torch.LongTensor([win[:context_size] + win[context_size + 1:]]).to(device)  # Context tokens (left and right)

        # Zero gradients
        opFoo.zero_grad()
        
        # Forward pass
        out = mFoo(inpt, trgs)
        
        # Assuming `out` is the output logits from the model
        true_index = torch.LongTensor([win[context_size]]).to(device)  # The index of the central token

        # Calculate the loss
        loss = criterion(out, true_index)
        
        # Backward pass and optimization
        loss.backward()
        opFoo.step()

        # Accumulate loss
        total_loss += loss.item()

        # Log the loss
        wandb.log({'loss': loss.item(), 'learning_rate': learning_rate})

    # Calculate and log average loss for the epoch
    average_loss = total_loss / len(wins) if len(wins) > 0 else 0
    wandb.log({'average_loss': average_loss})

# Finish the W&B logging
wandb.finish()

Epoch 1:   0%|          | 0/96 [00:00<?, ?it/s]

this is the shape inpt torch.Size([1])
this is the shape trgs torch.Size([1, 4])
this is the shape new inpt torch.Size([1, 1])
this is the shape torch.Size([1, 5])
shape of emb torch.Size([1, 5, 64])
shape of target words torch.Size([1, 5, 64])
shape of ctx torch.Size([5, 64])
similarity matrix is torch.Size([5, 5])
softmax matrix shape torch.Size([5, 5])
attention torch.Size([1, 5, 64])
show me what out looks like --> tensor([[ 0.5956, -0.6851, -0.0449,  ..., -0.2759, -0.6589,  0.4831]],
       grad_fn=<MmBackward0>)


Epoch 1:   3%|▎         | 3/96 [00:00<00:11,  7.89it/s]

this is the shape inpt torch.Size([1])
this is the shape trgs torch.Size([1, 4])
this is the shape new inpt torch.Size([1, 1])
this is the shape torch.Size([1, 5])
shape of emb torch.Size([1, 5, 64])
shape of target words torch.Size([1, 5, 64])
shape of ctx torch.Size([5, 64])
similarity matrix is torch.Size([5, 5])
softmax matrix shape torch.Size([5, 5])
attention torch.Size([1, 5, 64])
show me what out looks like --> tensor([[ 0.2715,  0.0564, -0.1109,  ..., -0.6076,  0.5311,  0.8267]],
       grad_fn=<MmBackward0>)
this is the shape inpt torch.Size([1])
this is the shape trgs torch.Size([1, 4])
this is the shape new inpt torch.Size([1, 1])
this is the shape torch.Size([1, 5])
shape of emb torch.Size([1, 5, 64])
shape of target words torch.Size([1, 5, 64])
shape of ctx torch.Size([5, 64])
similarity matrix is torch.Size([5, 5])
softmax matrix shape torch.Size([5, 5])
attention torch.Size([1, 5, 64])
show me what out looks like --> tensor([[ 0.0770,  0.4280, -0.7365,  ...,  0.0120, -0

Epoch 1:   5%|▌         | 5/96 [00:00<00:09,  9.20it/s]

...

Epoch 30:  98%|█████████▊| 94/96 [00:15<00:00,  6.40it/s]


this is the shape inpt torch.Size([1])
this is the shape trgs torch.Size([1, 4])
this is the shape new inpt torch.Size([1, 1])
this is the shape torch.Size([1, 5])
shape of emb torch.Size([1, 5, 64])
shape of target words torch.Size([1, 5, 64])
shape of ctx torch.Size([5, 64])
similarity matrix is torch.Size([5, 5])
softmax matrix shape torch.Size([5, 5])
attention torch.Size([1, 5, 64])
show me what out looks like --> tensor([[-3.9376, -3.6231, -3.8819,  ..., -3.5612, -3.5896, -3.4634]],
       grad_fn=<MmBackward0>)


Epoch 30: 100%|██████████| 96/96 [00:16<00:00,  6.05it/s]

this is the shape inpt torch.Size([1])
this is the shape trgs torch.Size([1, 4])
this is the shape new inpt torch.Size([1, 1])
this is the shape torch.Size([1, 5])
shape of emb torch.Size([1, 5, 64])
shape of target words torch.Size([1, 5, 64])
shape of ctx torch.Size([5, 64])
similarity matrix is torch.Size([5, 5])
softmax matrix shape torch.Size([5, 5])
attention torch.Size([1, 5, 64])
show me what out looks like --> tensor([[-3.7832, -3.5031, -3.7538,  ..., -3.5411, -3.6699, -3.9143]],
       grad_fn=<MmBackward0>)


average_loss,█▇▇▆▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
average_loss,0.04002
